<a href="https://colab.research.google.com/github/JedRoundy/Machine_Learning_For_Economists/blob/main/PSET_5/pset5coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import the modules and function you will use here
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
import warnings
warnings.filterwarnings('ignore')

This problem deals with regularized regression. The boston dataset is described right after it is loaded in just by running the code that is aleardy there.

In [2]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
print(housing['DESCR'])
X = pd.DataFrame(housing['data'], columns=housing['feature_names'])
y = pd.Series(housing['target'])
chosen_idx = np.random.choice(len(X.index), replace=False, size = 500)
X = X.iloc[chosen_idx]
y = y.iloc[chosen_idx]

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

$(a)$ Split the data into a train and a test set

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = .33)


$(b)$ Use this data to fit an OLS, LASSO, ridge, and ElasticNet model on the data. For now, use the default for the penalty coefficient. Display the coefficients and test error for each.

In [4]:
#Fit OLS object
ols = LinearRegression().fit(X_train, y_train)

#fit Lasso object
lasso = Lasso().fit(X_train, y_train)

#fit ridge object
ridge = Ridge().fit(X_train, y_train)

#fit elastic net object
net = ElasticNet().fit(X_train, y_train)

#Print out scores
print(f'OLS Score: {ols.score(X_test, y_test)}')
print(f'Lasso Score: {lasso.score(X_test, y_test)}')
print(f'Ridge Score: {ridge.score(X_test, y_test)}')
print(f'Elastic Net Score: {net.score(X_test, y_test)}')



coef_dict = {"Variable": X.columns, "OLS Coefficients": ols.coef_, "Lasso Coefficients": lasso.coef_, "Ridge Coefficients": ridge.coef_, "Elastic Net Coefficients": net.coef_}
coef_df = pd.DataFrame(data = coef_dict)

coef_df



OLS Score: 0.5035570648665049
Lasso Score: 0.1733248015946306
Ridge Score: 0.5029296900584876
Elastic Net Score: 0.26657282965722184


,Variable,OLS Coefficients,Lasso Coefficients,Ridge Coefficients,Elastic Net Coefficients
0,MedInc,0.325128,0.133449,0.323379,2.252263e-01
1,HouseAge,0.008782,0.002840,0.008849,8.340574e-03
2,AveRooms,0.006702,0.000000,0.010685,0.000000e+00
3,AveBedrms,0.243058,-0.000000,0.219326,-0.000000e+00
4,Population,0.000055,-0.000031,0.000055,9.437331e-07
5,AveOccup,-0.190954,-0.000000,-0.190347,-0.000000e+00
6,Latitude,-0.472886,-0.000000,-0.468297,-0.000000e+00
7,Longitude,-0.484649,-0.000000,-0.479457,-0.000000e+00


$(c)$ Describe the differences that you see in the coefficients and error. What is the cause of this difference in coefficients?

In [20]:
#It is clear that Lasso and Elastic Net have very similar outcomes in regard to coefficients and somewhat similar outcomes in regard to their test scores.

#Lasso and Net have both set some important variables to 0, causing them to poorly predict the test set.

#The reason of the difference in coefficients is simply because of their minimization problems. They are minimizing different values, and they will come up with different coefficients.

#It is interesting that Ridge and OLS found large values for longitude, but in opposite directions.


,Variable,OLS Coefficients,Lasso Coefficients,Ridge Coefficients,Elastic Net COefficients
0,MedInc,0.294304,0.079084,0.406837,0.206962
1,HouseAge,-0.003627,0.000000,0.001836,0.000000
2,AveRooms,-0.052960,0.000000,-0.054950,0.000000
3,AveBedrms,-0.207843,-0.000000,-0.481034,-0.000000
4,Population,0.000147,0.000111,0.000248,0.000177
5,AveOccup,0.019849,-0.000000,-0.040874,-0.000000
6,Latitude,14.481844,0.000000,2.466662,0.000000
7,Longitude,6.925351,-0.000000,-1.219396,-0.000000


$(d)$ Use K-fold cross validation to find an optimal penalty parameter for Ridge and Lasso.

In [20]:
alphas = np.arange(.01, 15, .01)

rl_params = {'alpha': alphas}

ridge_grid = GridSearchCV(ridge, rl_params, verbose = 1, cv = 4)

lasso_grid = GridSearchCV(lasso, rl_params, verbose = 1, cv = 4)

lasso_grid.fit(X_train, y_train)
ridge_grid.fit(X_train, y_train)


print(f'Best Lasso Alpha: {lasso_grid.best_estimator_}')
print('\n')
print(f'Best Ridge Alpha: {ridge_grid.best_estimator_}')


Fitting 4 folds for each of 1499 candidates, totalling 5996 fits
Fitting 4 folds for each of 1499 candidates, totalling 5996 fits
Best Lasso Alpha: Lasso(alpha=0.02)


Best Ridge Alpha: Ridge(alpha=14.42)


$(e)$ Now use cross validation, to find the optimal penalty parameter. Use LOOCV and Kfold cross validation with K=5 to find optimal parameters for the ElasticNet model. How do the test errors and optimal parameters differ?

In [33]:
alphas = np.arange(.1, 1, .1)

l1 = np.arange(0, 1, .1)

net_params = {"alpha":alphas, 'l1_ratio':l1}

LOOCV_net = GridSearchCV(net, net_params, cv = len(X_train), verbose = 1)
kfold_net = GridSearchCV(net, net_params, cv = 5, verbose = 1)

LOOCV_net.fit(X_train, y_train)
kfold_net.fit(X_train, y_train)


print(f'LOOCV best score:  {LOOCV_net.best_score_}')
print(f'K-Fold best score:  {kfold_net.best_score_}')
print('\n')

print(f'LOOCV Parameters: { LOOCV_net.best_estimator}')
print(f'K-Fold Parameters: { kfold_net.best_estimator}')



Fitting 300 folds for each of 90 candidates, totalling 27000 fits


ValueError: ignored

$(f)$ Now that we have tuned the models to perform about as well as they can, which one performs best on the training data? Which one performs best on the test data? Which of these models allow us to do effective causal inference with the coefficients? Why?

In [38]:

print(f'Ridge Training Score: {ridge_grid.score(X_train, y_train)}')
print(f'Lasso Training Score: {lasso_grid.score(X_train, y_train)}')
print(f'Elastic Net LOOCV Training Score: {LOOCV_net.score(X_train, y_train)}')
print(f'Elastic Net K-Folds Training Score: {kfold_net.score(X_train, y_train)}')
print('\n')

print(f'Ridge Test Score: {ridge_grid.score(X_test, y_test)}')
print(f'Lasso Test Score: {lasso_grid.score(X_test, y_test)}')
print(f'Elastic Net LOOCV Test Score: {LOOCV_net.score(X_test, y_test)}')
print(f'Elastic Net K-Folds Test Score: {kfold_net.score(X_test, y_test)}')

#Surprisingly, they all perform better on the test set than on the training set.

Ridge Training Score: 0.6946367528950361
Lasso Training Score: 0.694681220159955
Elastic Net LOOCV Training Score: 0.694681220159955
Elastic Net K-Folds Training Score: 0.694681220159955


Ridge Test Score: 0.7711024428273829
Lasso Test Score: 0.7707796217109142
Elastic Net LOOCV Test Score: 0.7707796217109142
Elastic Net K-Folds Test Score: 0.7707796217109142


For the next problem we will be using the `Carseats` data set that is available on learningsuite. Load the data and convert the text variables into dummies so that we can use them in the data. Pandas has a function called `get_dummies` that you might want to use.

In [55]:
carseats = pd.read_csv('https://raw.githubusercontent.com/JedRoundy/Machine_Learning_For_Economists/main/data/pset_5/Carseats.csv')

carseats = pd.get_dummies(carseats)

Now that the data has only numeric columns, we can proceed to the analysis.  
Use `Sales` as the outcome variable  
(a) Split the data set into a training set and a test set.  
(b) Fit a regression tree to the training set with the default depth. What train and test MSE do you obtain?  
(c) Use cross-validation in order to determine the optimal level of tree complexity. Does pruning the tree improve the test MSE? Plot a tree with a depth of 3, and interpret the results.  
(d) Use a bagging approach in order to analyze this data. What test MSE do you obtain? Look at the feature importances attribute of your model object to determine which variables are most important.  
(e) Use random forests to analyze this data. What test MSE do you obtain? Look at the feature importances attribute of your model object function to determine which variables are most important. Describe the effect of m, the number of variables considered at each split, on the error rate obtained.

In [59]:
from sklearn.tree import DecisionTreeRegressor, export_graphviz
import graphviz

y = carseats.filter(['Sales'])
X = carseats.drop('Sales', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y)

dtr = DecisionTreeRegressor(random_state = 0)

dtr.fit(X_train, y_train)
print(f'Regression Tree Train Score: {dtr.score(X_train, y_train)}')
print(f'Regression Tree Test Score: {dtr.score(X_test, y_test)}')

params = {'max_depth': [2,3,4,5,6,7,8], 'min_samples_split': [5, 10, 15, 20], 'random_state': [0]}

grid = GridSearchCV(dtr, param_grid = params)

grid.fit(X_train, y_train)

print('\n')
print(f'GSCV Regression Tree Params: {grid.get_params}')
print(f'GSCV Regression Tree Train Score: {grid.score(X_train, y_train)}')
print(f'GSCV Regression Tree Test Score: {grid.score(X_test, y_test)}')

#Yes, pruning the tree improves test score

clf = DecisionTreeRegressor(max_depth = 3).fit(X_train, y_train)
# Visualize the tree
dot_data = export_graphviz(clf, out_file=None, feature_names=X.columns,
                           class_names=y.columns, filled=True)
graph = graphviz.Source(dot_data)
graph



GSCV Regression Tree Params: <bound method BaseEstimator.get_params of GridSearchCV(estimator=DecisionTreeRegressor(random_state=0),
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7, 8],
                         'min_samples_split': [5, 10, 15, 20],
                         'random_state': [0]})>


We will now use boosting to predict Log Salary in the `Hitters` data set.  
(a) Format the data appropriately for this analysis. Use 200 observations in your training set.  
(b) Perform boosting on the training set with 1,000 trees for a range of values of the shrinkage parameter λ. Produce a plot with different shrinkage values on the x-axis and the corresponding training set MSE on the y-axis. Add a curve with different shrinkage values on the x-axis and the corresponding test set MSE on the y-axis. The shrinkage parameter is often referred to as the learning rate   
(c) Compare the test MSE of boosting to the test MSE of two of the penalized regression approaches that we discussed  
(d) Which variables appear to be the most important predictors in the boosted model?  
(e) The default for base estimator is a Decision Tree with a maximum depth of 3. Is that the optimal depth? Justify your response.  
(f) Now that the boosting model is tuned, let's compare the results to bagging and random forests. Report test errors for your models and discuss how they compare.

In [ ]:
hitters = pd.read_csv('https://raw.githubusercontent.com/JedRoundy/Machine_Learning_For_Economists/main/data/pset_5/Hitters.csv')

In this problem, you will use support vector approaches in order to predict whether a given car gets high or low gas mileage based on the Auto data set.  

#### NOTE: SVM algortihms will often take longer than other models to train, particularly when doing cross validation

(a) Create a binary variable that takes on a 1 for cars with gas mileage above the median, and a 0 for cars with gas mileage below the median.  
(b) Fit a support vector classifier to the data with various values of cost, in order to predict whether a car gets high or low gas mileage. Report the cross-validation errors associated with different values of this parameter. Comment on your results.  
(c) Make an ROC curve for your model. The module scikitplot has a nice function you might want to use but you should eb able to make it on your own or another module if you desire.

Below there are some generated datasets of varying structure that you will classifying is SVMs, plotting the data to see what it looks like will likey be helpful. Find the basis kernel that does best job classifying each of them. Because the data is two dimensional, it might be nice to use a library like mlxtend which has a function that will display decision regions form an svm using a one of their functions.

In [ ]:
from sklearn.datasets import make_moons
x, y = make_moons(n_samples=100, shuffle=True, noise=1/10, random_state=123)

In [ ]:
from sklearn.datasets import make_circles
x, y = make_circles(n_samples=100, shuffle=False, noise=1/50, random_state=123, factor=0.6)

In [ ]:
from sklearn.datasets import make_blobs
x, y = make_blobs(n_samples=100, n_features=2, centers=None, cluster_std=2.0,
           center_box=(-10.0, 10.0), shuffle=True, random_state=10)